In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import os
import torch
import tqdm.notebook as tqdm

from sklearn.metrics import confusion_matrix, classification_report
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Patch
from shapely import wkt
# from src.models.model_utils import *
from src.data.ee_utils import *
from src.data.data_utils import *
from src.data.dataset import *
from src.data.classes import *

# LUCAS2018_LEVEL2_CODES['NOT_CROP'] = 'Not Crop'

/u/ayang1/.conda/envs/CropTS/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [35]:
ts_5_data = process_fused_data('/scratch/bbug/ayang1/datasets/lucas_fused_10/csvs')

fused_Bulgaria_Lovech.csv
fused_Germany_Baden-Wuerttemberg.csv
fused_Spain_Castilla-La Mancha_Toledo.csv
fused_data.csv
fused_Romania_Vilcea.csv
fused_Luxembourg.csv
fused_Estonia_Saare maakond_Orissaare vald.csv
fused_Lithuania.csv
fused_Estonia_Saare maakond_Kuressaare linn.csv
fused_Poland_Dolnoslaskie.csv
fused_Bulgaria_Ruse.csv
fused_Portugal_Beja.csv
fused_Greece_Voreio Aigaio.csv
fused_Italy_Valle D'aosta.csv
fused_Estonia_Saare maakond_Muhu vald.csv
fused_Estonia_Jõgeva maakond.csv
fused_Poland_Lubuskie.csv
fused_Estonia_Järva maakond.csv
fused_Poland_Zachodnio-Pomorskie.csv
fused_Ireland.csv
fused_Romania_Braila.csv
fused_Spain_Castilla y León_Soria.csv
fused_Romania_Caras-severin.csv
fused_Romania_Mures.csv
fused_Estonia_Pärnu maakond_Kihnu vald.csv
fused_Italy_Marche.csv
fused_Spain_Cantabria.csv
fused_Estonia_Pärnu maakond_Tori vald.csv
fused_Italy_Emilia-romagna.csv
fused_Romania_Iasi.csv
fused_Spain_Castilla y León_Ávila.csv
fused_Austria_Wien.csv
fused_Sweden_Blekinge La

In [37]:
path = '/scratch/bbug/ayang1/datasets/lucas_fused_10'
data = pd.read_csv('/scratch/bbug/ayang1/datasets/lucas_fused_10/csvs/fused_data.csv')

In [28]:
import geopandas as gpd
data['geometry'] = data['geo'].apply(lambda x: ast.literal_eval(x))
data['geometry'] = data['geometry'].apply(lambda sample: wkt.loads(f"{sample['type'].upper()} ({sample['coordinates'][0]} {sample['coordinates'][1]})")
# Convert dataframe to geodataframe
# gdf = gpd.GeoDataFrame(data, geometry='geometry')

# Save as shapefile
# gdf.to_file('data.shp')

POINT (-9.35860371419297 39.07819707941798)


In [32]:
np.unique([len(x['coordinates']) for x in data['geometry']],return_counts=True)

(array([2]), array([58375]))